In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb

random_state = 1

In [2]:
def euclid(list1, list2):
    return sum((p-q)**2 for p, q in zip(list1, list2)) ** .5

In [3]:
with open('df_policies_embeddings.pkl', 'rb') as f:
    df = pickle.load(f)
df

,name,text,Level,document_name,Type,embeddings
0,Category,ACCOUNT INFORMATION,None,Fitbit.txt,Closed,"[1.376953125, -1.1220703125, 1.982421875, 1.12..."
1,SubCategory,your name,None,Fitbit.txt,Closed,"[0.276123046875, 0.7734375, 0.53076171875, 1.2..."
2,SubCategory,email address,None,Fitbit.txt,Closed,"[1.5146484375, -0.07098388671875, 0.3347167968..."
3,SubCategory,password,None,Fitbit.txt,Closed,"[0.436279296875, -0.021484375, 0.6201171875, -..."
4,SubCategory,date of birth,None,Fitbit.txt,Closed,"[0.5732421875, -1.681640625, -0.114990234375, ..."
...,...,...,...,...,...,...
4113,Category,your contact information,None,Snapchat.txt,Closed,"[1.708984375, -0.94384765625, 0.984375, 0.6762..."
4114,SubCategory,information from third parties,None,Snapchat.txt,Open,"[0.6962890625, -1.0859375, 0.611328125, 0.6069..."
4115,cat,"Or, if you provide us with your contact inform...",2,Snapchat.txt,None,"[0.485107421875, -2.080078125, 0.92724609375, ..."
4116,cat,"Other Information, With Your Permission",2,Snapchat.txt,None,"[0.6630859375, -1.1650390625, 0.44580078125, 1..."


In [4]:
validation_set = ['X_Twitter.txt', 'LinkedIn.txt', 'Zoom.txt', 'Fitbit.txt', 'Western Union.txt', 'Ubisoft.txt']
test_set =  ['Yahoo.txt', 'TikTok.txt', 'MySugr.txt', 'Deliveroo.txt', 'Paypal.txt', 'AdobePhotoshop.txt']
nontrain_set = validation_set + test_set
nontrain_set

['X_Twitter.txt',
 'LinkedIn.txt',
 'Zoom.txt',
 'Fitbit.txt',
 'Western Union.txt',
 'Ubisoft.txt',
 'Yahoo.txt',
 'TikTok.txt',
 'MySugr.txt',
 'Deliveroo.txt',
 'Paypal.txt',
 'AdobePhotoshop.txt']

In [5]:
df_test = df[df.document_name.isin(test_set)]
df_test = df_test.dropna(subset = ['Level'])
df_test

,name,text,Level,document_name,Type,embeddings
492,cat,We collect information that you provide when y...,1,TikTok.txt,None,"[1.44921875, -0.89892578125, 1.8203125, 0.1143..."
496,cat,User Content.,2,TikTok.txt,None,"[0.123291015625, -1.8173828125, 1.369140625, 1..."
498,cat,We collect the content you create or publish t...,2,TikTok.txt,None,"[-0.5400390625, -1.1865234375, 2.10546875, 0.4..."
508,cat,"We also collect content (such as text, images,...",2,TikTok.txt,None,"[-0.35302734375, -1.4091796875, 1.4365234375, ..."
509,cat,"In addition, we collect location information (...",1,TikTok.txt,None,"[0.58251953125, -0.2344970703125, 2.423828125,..."
...,...,...,...,...,...,...
3564,cat,Information about your imported contacts.,2,Paypal.txt,None,"[0.5029296875, -1.8271484375, 0.68701171875, 1..."
3575,cat,Information you disclose when you respond to s...,1,Paypal.txt,None,"[0.62158203125, -2.248046875, 1.111328125, 0.2..."
3577,cat,This may include information about others if y...,1,Paypal.txt,None,"[0.7802734375, -1.66796875, -0.4091796875, 0.7..."
3586,cat,"For example, if you choose to participate in O...",2,Paypal.txt,None,"[1.154296875, -0.71728515625, -0.791015625, 0...."


In [6]:
device = torch.device('cuda')
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [7]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16,
                  "quantization_config": {"load_in_4bit": True}},
    device_map="cuda:0"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [8]:
template = '''You will be given as input a sentence from a privacy policy that contains information about what data the service collects about the user. You have to classify the sentence into one of the following classes: "sufficiently informative" or "insufficiently informative".

In doing so, consider that GDPR, and the EDPB's Guidelines, contain a certain inherent tension, namely that between requiring that the information is provided in as easy a way to understand as possible (comprehensibility) and that it is concrete and definitely (comprehensiveness).
Sometimes, using open-ended qualifiers like “for example” or “such as” might actually facilitate understanding by the data subject, especially when terms not often used in the natural language (e.g. “device information” or “geolocation information”) are concerned. For this reason, we differentiate between:
*	Abstract terms (e.g. usage information) vs. concrete terms (e.g. geolocation information)
*	Open-ended qualifications (“for example”, “such as”, etc.) vs. closed-catalogues (“meaning”, “understood as” ).

A sentence containing an abstract term, UNLESS followed by a comprehensive enumeration, should be judged as "insufficiently informative".
A sentence containing a concrete term, even if followed by an open-ended qualifier, should be judged as "sufficiently informative".

Avoid explanations. Only reply with "insufficiently informative" or "sufficiently informative".'''

In [9]:
def test(template, nexamples):

    count = 0
    LV1TP = 0
    LV1FN = 0
    LV1FP = 0
    LV2TP = 0
    LV2FN = 0
    LV2FP = 0

    for i in range(len(df_test)):

        query = 'Text: '+ df_test.iloc[i]['text']+'\n'+'label = '
        
        print('query:\n'+ query,'\n')

        messages = [{'role': 'system', 'content': template},{"role": "user", "content": query}]

        outputs = pipeline(
            messages,
            max_new_tokens=10,
            do_sample = False
        )
        final_output = outputs[0]['generated_text'][-1]['content']
        print('LLM output:', final_output)
        if df_test.iloc[i]['Level'] == '1':
            gt = 'sufficiently informative'
        else:
            gt = 'insufficiently informative'
        print('ground truth:', gt)
        print(200*'-')

        if 'insufficiently' in final_output and df_test.iloc[i]['Level'] == '2':
            LV2TP += 1
        if 'insufficiently' in final_output and df_test.iloc[i]['Level'] == '1':
            LV2FP += 1
        if 'insufficiently' not in final_output and df_test.iloc[i]['Level'] == '2':
            LV2FN += 1
        
        if  final_output == 'sufficiently informative' and df_test.iloc[i]['Level'] == '1':
            LV1TP += 1
        if final_output == 'sufficiently informative' and df_test.iloc[i]['Level'] == '2':
            LV1FP += 1
        if final_output != 'sufficiently informative' and df_test.iloc[i]['Level'] == '1':
            LV1FN += 1

    LV2PREC = LV2TP/(LV2TP+LV2FP)
    LV2REC = LV2TP/(LV2TP+LV2FN)
    LV1PREC = LV1TP/(LV1TP+LV1FP)
    LV1REC = LV1TP/(LV1TP+LV1FN)

    LV2F1 = 2/((1/LV2PREC)+(1/LV2REC))
    LV1F1 = 2/((1/LV1PREC)+(1/LV1REC))

    print('lv2 F1:', LV2F1)
    print('lv1 F1:', LV1F1)

    print('macro F1:', (LV2F1+LV1F1)/2 )
            


In [10]:
test(template, 5)

C:\Users\alfio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\alfio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


query:
Text: We collect information that you provide when you set up an account, such as your date of birth, username, email address and/or telephone number, and password.
label =  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM output: sufficiently informative
ground truth: sufficiently informative
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: User Content.
label =  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

LLM output: sufficiently informative
ground truth: sufficiently informative
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: This may include information about others if you choose to share it with us.
label =  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM output: insufficiently informative
ground truth: sufficiently informative
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: For example, if you choose to participate in Open Banking, we will collect account credentials, account balances, account transactions, and
information about your financial standing from your linked accounts
label =  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM output: sufficiently informative
ground truth: insufficiently informative
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: This optional data is collected and processed in order to store and retain necessary financial information in case you make a purchase, as well as to improve our app experience for you.
label =  

LLM output: sufficiently informative
ground truth: insufficiently informative
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
lv2 F1: 0.3552631578947369
lv1 F1: 0.42944785276073627
macro F1: 0.39235550532773655
